In [ ]:
import torch
import pandas as pd
import gc
from tqdm.auto import tqdm, trange
import numpy as np

from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score
import torch.nn as nn

def cleanup():
    gc.collect()
    torch.cuda.empty_cache()

# Загружаем данные
df = pd.read_excel(r"results\treaning.xlsx")

text_col = "text"
# Предполагаем, что столбцы для меток должны быть числовыми
category_cols = [col for col in df.columns if col != text_col]
df[category_cols] = df[category_cols].apply(pd.to_numeric, errors='coerce').fillna(0)
df["labels"] = df[category_cols].values.tolist()
df.dropna(subset=[text_col], inplace=True)

# Преобразуем в Dataset
data = Dataset.from_pandas(df)
train_dataset, test_dataset = data.train_test_split(test_size=0.2).values()

# Устанавливаем max_length
max_length = 512
tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruRoberta-large")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def tokenize_function(examples):
    tokenized = tokenizer(
        examples[text_col],
        truncation=True,
        padding="max_length",
        max_length=max_length
    )
    float_labels = []
    for row in examples["labels"]:
        float_labels.append([float(x) for x in row])
    tokenized["labels"] = float_labels
    return tokenized

cols_to_remove = [text_col] + category_cols + ["__index_level_0__"]

train_dataset = train_dataset.map(
    tokenize_function, 
    batched=True, 
    remove_columns=cols_to_remove
)
test_dataset = test_dataset.map(
    tokenize_function, 
    batched=True, 
    remove_columns=cols_to_remove
)

batch_size = 8
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=data_collator)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=data_collator)

num_labels = len(category_cols)
model = AutoModelForSequenceClassification.from_pretrained(
    "ai-forever/ruRoberta-large",
    num_labels=num_labels,
    problem_type='multi_label_classification'
).cuda()

# Вычисляем веса для функции потерь с учётом дисбаланса классов
all_labels_train = np.array(train_dataset["labels"])
positive_counts = all_labels_train.sum(axis=0)
negative_counts = all_labels_train.shape[0] - positive_counts
eps = 1e-5
pos_weight = (negative_counts + eps) / (positive_counts + eps)
pos_weight = torch.tensor(pos_weight, dtype=torch.float32).cuda()

loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

def evaluate_model(model, dataloader, thresholds=None):
    model.eval()
    all_preds, all_labels = [], []
    for batch in tqdm(dataloader, desc="Evaluating"):
        labels = batch['labels'].cpu().numpy()
        batch = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
        with torch.no_grad():
            logits = model(**batch).logits
        probs = torch.sigmoid(logits).cpu().numpy()
        if thresholds is None:
            preds = (probs > 0.5).astype(int)
        else:
            preds = np.zeros_like(probs)
            for i, t in enumerate(thresholds):
                preds[:, i] = (probs[:, i] > t).astype(int)
        all_preds.extend(preds)
        all_labels.extend(labels)
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    exact_match = np.mean(np.all(all_preds == all_labels, axis=1))
    f1 = f1_score(all_labels, all_preds, average='samples')
    return exact_match, f1

def calibrate_thresholds(model, dataloader):
    model.eval()
    all_logits, all_labels = [], []
    for batch in tqdm(dataloader, desc="Calibrating thresholds"):
        labels = batch['labels'].cpu().numpy()
        batch = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
        with torch.no_grad():
            logits = model(**batch).logits
        all_logits.extend(logits.cpu().numpy())
        all_labels.extend(labels)
    all_logits = np.array(all_logits)
    all_labels = np.array(all_labels)
    optimal_thresholds = []
    num_labels = all_labels.shape[1]
    for i in range(num_labels):
        best_threshold = 0.5
        best_f1 = 0
        for threshold in np.arange(0.1, 0.9, 0.05):
            probs = 1 / (1 + np.exp(-all_logits[:, i]))
            preds = (probs > threshold).astype(int)
            f1 = f1_score(all_labels[:, i], preds, zero_division=0)
            if f1 > best_f1:
                best_f1 = f1
                best_threshold = threshold
        optimal_thresholds.append(best_threshold)
    return optimal_thresholds

# Гиперпараметры обучения
num_epochs = 10  # увеличено число эпох
gradient_accumulation_steps = 1
cleanup_step = 100
report_step = 50
ewm_loss = 0
window = 500

# Настраиваем оптимизатор и scheduler
learning_rate = 2e-5  # можно варьировать
optimizer = torch.optim.AdamW(params=model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(0.1*total_steps), num_training_steps=total_steps)

# Ранняя остановка: если F1 на валидации не улучшается в течение patience эпох, останавливаем обучение
patience = 3
best_f1 = 0
epochs_without_improvement = 0

# На первом этапе можно заморозить базовые слои, а затем разморозить их
for param in model.roberta.parameters():
    param.requires_grad = False

for epoch in trange(num_epochs, desc="Epochs"):
    # Если это не первый эпоh, размораживаем базовые слои
    if epoch > 0:
        for param in model.roberta.parameters():
            param.requires_grad = True

    model.train()
    cleanup()
    tq = tqdm(train_dataloader, desc="Training")
    for i, batch in enumerate(tq):
        batch = {k: v.to(model.device) for k, v in batch.items()}
        batch['labels'] = batch['labels'].float()  # преобразуем метки в float
        outputs = model(**batch)
        logits = outputs.logits
        loss = loss_fn(logits, batch['labels'])
        loss.backward()
        
        if (i + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            scheduler.step()  # обновляем lr
            optimizer.zero_grad()
        
        if i % cleanup_step == 0:
            cleanup()
        
        w = 1 / min(i + 1, window)
        ewm_loss = ewm_loss * (1 - w) + loss.item() * w
        tq.set_description(f'loss: {ewm_loss:.4f}')
        
        if i % report_step == 0 and i > 0:
            eval_acc, eval_f1 = evaluate_model(model, test_dataloader)
            print(f'epoch {epoch}, step {i}: train loss: {ewm_loss:.4f}, exact match acc: {eval_acc:.4f}, F1: {eval_f1:.4f}')
    
    # Оценка модели после каждой эпохи
    eval_acc, eval_f1 = evaluate_model(model, test_dataloader)
    print(f'Epoch {epoch} завершена: exact match acc: {eval_acc:.4f}, F1: {eval_f1:.4f}')
    
    # Ранняя остановка: если улучшение F1 отсутствует, увеличиваем счётчик
    if eval_f1 > best_f1:
        best_f1 = eval_f1
        epochs_without_improvement = 0
        # Сохраняем лучшую модель
        model.save_pretrained("./best_ruRoberta_multilabel_model")
        tokenizer.save_pretrained("./best_ruRoberta_multilabel_model")
    else:
        epochs_without_improvement += 1
        print(f"Улучшение F1 не наблюдалось в течение {epochs_without_improvement} эпох.")
        if epochs_without_improvement >= patience:
            print("Ранняя остановка обучения.")
            break

optimal_thresholds = calibrate_thresholds(model, test_dataloader)
print("Optimal thresholds for each category:", optimal_thresholds)

model.eval()
model.save_pretrained("./final_ruRoberta_multilabel_model")
tokenizer.save_pretrained("./final_ruRoberta_multilabel_model")


In [ ]:
import pandas as pd
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from tqdm import tqdm
import time

# 1. Считываем отзывы из Excel-файла
input_file = r"results\reviews.xlsx"  # Название входного файла Excel
df = pd.read_excel(input_file)

# Если колонки "Классификация" ещё нет, добавляем её; если есть, заполняем пустыми значениями
if "Классификация" not in df.columns:
    df["Классификация"] = ""
else:
    df["Классификация"] = df["Классификация"].fillna("")

# Определяем индексы отзывов, у которых ещё нет классификации (пустое значение в колонке "Классификация")
pending_indices = df.index[df["Классификация"] == ""].tolist()
pending_reviews = df.loc[pending_indices, "Отзыв"].tolist()

# 2. Список категорий (используется лишь для определения порядка меток)
label_names = [
    "Материальный аспект (позитивный)",
    "Материальный аспект (негативный)",
    "Аспект безопасности (позитивный)",
    "Аспект безопасности (негативный)",
    "Аспект эффективности (позитивный)",
    "Аспект эффективности (негативный)",
    "Межличностный аспект (позитивный)",
    "Межличностный аспект (негативный)"
]

# 3. Определяем устройство: cuda, если доступна, иначе cpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Используем устройство:", device)

# 4. Загрузка модели и токенизатора для Roberta и перемещение модели на устройство
model_path = "./final_ruRoberta_multilabel_model"  # Убедитесь, что здесь находится модель на основе Roberta
tokenizer = RobertaTokenizer.from_pretrained(model_path)
model = RobertaForSequenceClassification.from_pretrained(model_path)
model.to(device)
model.eval()

# Порог для определения активности метки
threshold = 0.7

# 5. Обработка отзывов, у которых отсутствует классификация, пакетами с отображением ETA
batch_size = 16  # Размер батча можно регулировать
n_pending = len(pending_reviews)
start_overall = time.time()

for i in tqdm(range(0, n_pending, batch_size), desc="Классификация отзывов", unit="batch"):
    batch_reviews = pending_reviews[i: i + batch_size]
    
    # Токенизация отзывов с переносом на устройство
    inputs = tokenizer(
        batch_reviews,
        truncation=True,
        padding=True,
        max_length=512,
        return_tensors="pt"
    ).to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
    
    # Получаем бинарные предсказания для каждой метки
    predictions = (torch.sigmoid(logits) > threshold).int().cpu().numpy()
    
    # Обновляем DataFrame для каждой обработанной записи, записывая строку из 8 цифр через запятую
    for j, pred in enumerate(predictions):
        pred_str = ",".join(map(str, pred.tolist()))
        orig_idx = pending_indices[i + j]
        df.at[orig_idx, "Классификация"] = pred_str
    
    # Расчет оставшегося времени (ETA)
    processed = min(i + batch_size, n_pending)
    elapsed = time.time() - start_overall
    avg_time = elapsed / processed if processed > 0 else 0
    remaining_reviews = n_pending - processed
    remaining_time = remaining_reviews * avg_time


# 6. Сохраняем обновленный DataFrame в новый Excel-файл
output_file = "classified_reviews_prodoctorov.xlsx"
df.to_excel(output_file, index=False)
print("Классификация завершена. Результаты записаны в файл:", output_file)


Используем устройство: cuda


Классификация отзывов: 100%|██████████| 75/75 [00:27<00:00,  2.71batch/s]


Классификация завершена. Результаты записаны в файл: classified_reviews_prodoctorov.xlsx


In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm
import time

# 1. Считываем отзывы из Excel-файла
input_file = r"results\reviews.xlsx"  # Файл с отзывами. Предполагается, что текст отзыва находится в колонке "Отзыв"
df = pd.read_excel(input_file)

# Если колонки "Эмоции" ещё нет, создаём её; если есть – заполняем NaN пустыми строками
if "Эмоции" not in df.columns:
    df["Эмоции"] = ""
else:
    df["Эмоции"] = df["Эмоции"].fillna("")

# Определяем индексы отзывов, для которых ещё не выполнен анализ эмоций
pending_indices = df.index[df["Эмоции"] == ""].tolist()
pending_reviews = df.loc[pending_indices, "Отзыв"].tolist()

# 2. Параметры модели эмоций
best_thresholds = [0.36734693877551017, 0.2857142857142857, 0.2857142857142857, 0.16326530612244897, 
                   0.14285714285714285, 0.14285714285714285, 0.18367346938775508, 0.3469387755102041, 
                   0.32653061224489793, 0.22448979591836732, 0.2040816326530612, 0.2857142857142857, 
                   0.18367346938775508, 0.2857142857142857, 0.24489795918367346, 0.7142857142857142, 
                   0.02040816326530612, 0.3061224489795918, 0.44897959183673464, 0.061224489795918366, 
                   0.18367346938775508, 0.04081632653061224, 0.08163265306122448, 0.1020408163265306, 
                   0.22448979591836732, 0.3877551020408163, 0.3469387755102041, 0.24489795918367346]

LABELS = ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 
          'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 
          'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 
          'remorse', 'sadness', 'surprise', 'neutral']
ID2LABEL = dict(enumerate(LABELS))

# 3. Определяем устройство: если доступна CUDA, используем GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Используем устройство:", device)

# Загрузка токенизатора и модели для анализа эмоций
model_name = "fyaronskiy/ruRoberta-large-ru-go-emotions"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.to(device)
model.eval()

# Приводим best_thresholds к тензору и перемещаем на устройство
best_thresholds_tensor = torch.tensor(best_thresholds).to(device)

# 4. Обработка отзывов пакетами с отображением оставшегося времени (ETA)
batch_size = 16  # Можно скорректировать размер батча
n_pending = len(pending_reviews)
start_overall = time.time()

for i in tqdm(range(0, n_pending, batch_size), desc="Анализ эмоций", unit="batch"):
    batch_reviews = pending_reviews[i: i + batch_size]
    
    # Токенизируем отзывы (для каждого батча)
    inputs = tokenizer(
        batch_reviews,
        truncation=True,
        add_special_tokens=True,
        max_length=128,
        padding=True,
        return_tensors="pt"
    ).to(device)
    
    with torch.no_grad():
        logits = model(**inputs).logits  # shape: (batch_size, 28)

    # Вычисляем вероятность для каждой эмоции (сигмоида)
    probas = torch.sigmoid(logits)  # на устройстве, shape: (batch_size, 28)
    # Применяем порог для получения бинарных меток по каждому классу
    predictions = (probas > best_thresholds_tensor).int().cpu().numpy()  # возвращаем результат в CPU

    # Обновляем DataFrame: формируем для каждого отзыва строку с предсказанными эмоциями
    for j, pred in enumerate(predictions):
        emotions = [ID2LABEL[label_id] for label_id, value in enumerate(pred) if value == 1]
        # Если ни одна эмоция не предсказана – можно записать, например, "neutral" или пустую строку.
        emotions_str = ", ".join(emotions) if emotions else ""
        orig_idx = pending_indices[i + j]
        df.at[orig_idx, "Эмоции"] = emotions_str

    # Вычисление оставшегося времени (ETA)
    processed = min(i + batch_size, n_pending)
    elapsed = time.time() - start_overall
    avg_time = elapsed / processed if processed > 0 else 0
    remaining_reviews = n_pending - processed
    remaining_time = remaining_reviews * avg_time


# 5. Сохраняем обновленный DataFrame в новый Excel-файл
output_file = "classified_reviews_prodoctorov.xlsx"
df.to_excel(output_file, index=False)
print("Анализ эмоций завершён. Результаты записаны в файл:", output_file)

Используем устройство: cuda


Анализ эмоций: 100%|██████████| 77/77 [00:12<00:00,  5.94batch/s]


Анализ эмоций завершён. Результаты записаны в файл: classified_reviews_prodoctorov.xlsx


In [ ]:
import pandas as pd

# Определяем множества положительных и отрицательных эмоций
positive_emotions = {
    'admiration', 'amusement', 'approval', 'caring',
    'excitement', 'gratitude', 'joy', 'love',
    'optimism', 'pride', 'relief'
}
negative_emotions = {
    'anger', 'annoyance', 'disappointment', 'disapproval',
    'disgust', 'embarrassment', 'fear', 'grief',
    'nervousness', 'remorse', 'sadness'
}

# Загрузка Excel-файла (замените 'your_file.xlsx' на ваш файл)
input_file = r"results\reviews.xlsx"  # Файл с отзывами. Предполагается, что текст отзыва находится в колонке "Отзыв"
df = pd.read_excel(input_file)


def classify_emotions(emotion_str):
    """
    Функция разбивает строку эмоций (разделенных запятыми) 
    и возвращает кортеж (положительная_эмоция, отрицательная_эмоция).
    
    Если значение пустое или равно "neutral", возвращает (0, 0).
    """
    # Если значение пустое или NaN
    if pd.isna(emotion_str) or not str(emotion_str).strip():
        return 0, 0

    # Разбиваем строку по запятой и приводим к нижнему регистру
    emotions = [e.strip().lower() for e in emotion_str.split(',')]
    
    # Если присутствует только значение "neutral"
    if len(emotions) == 1 and emotions[0] == 'neutral':
        return 0, 0
    
    # Проверяем, есть ли среди эмоций положительная и отрицательная
    positive_flag = 1 if any(e in positive_emotions for e in emotions) else 0
    negative_flag = 1 if any(e in negative_emotions for e in emotions) else 0

    return positive_flag, negative_flag

# Применяем функцию к колонке "Эмоции"
df[['Положительные_эмоции', 'Отрицательные_эмоции']] = df['Эмоции'].apply(
    lambda x: pd.Series(classify_emotions(x))
)

# Вывод первых строк полученного DataFrame
print(df.head())

# Сохраняем результат в новый Excel-файл (при необходимости)
df.to_excel('Продокторов_полный.xlsx', index=False)


  web-scraper-order                                              Отзыв  score  \
0      1746816383-1  Операция была 19 сентября 2017 года. Оперирова...    5.0   
1      1746816383-2  Выражаю глубокие слова благодарности медицинск...    5.0   
2      1746816383-3  Новокшонову Константину Юрьевичу. Большое спас...    5.0   
3      1746816383-4  Я, Лифшиц Нина Аркадьевна, 89 лет, Санкт-Петер...    5.0   
4      1746816383-5  15.08.2017 моей жене была проведена операция п...    5.0   

                                      date    Классификация  \
0  27 сентября 2017\n\n            в 17:32  1,0,1,0,1,0,1,0   
1  22 сентября 2017\n\n            в 22:37  1,0,1,0,1,0,1,0   
2  19 сентября 2017\n\n            в 18:33  0,0,0,0,1,0,1,0   
3  11 сентября 2017\n\n            в 13:31  0,0,1,0,1,0,1,0   
4  11 сентября 2017\n\n            в 10:14  0,0,1,0,1,0,1,0   

   Материальный аспект (позитивный)  Материальный аспект (негативный)  \
0                                 1                          

In [ ]:
# Список отзывов для классификации
reviews = [
    "Впечатляет наличие современного оборудования и высоких технологий, которые помогли быстро поставить диагноз.",
    "Долгие очереди и некомпетентный персонал оставили неприятное впечатление.",
    "Прекрасный ремонт, чистота помещений и дружелюбный персонал – рекомендую!",
    "Современная клиника и эффективное лечение, однако персонал удивил невниманием.",
]

import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import pandas as pd

# Прямой список названий категорий
label_names = [
    "Материальный аспект (позитивный)",
    "Материальный аспект (негативный)",
    "Аспект безопасности (позитивный)",
    "Аспект безопасности (негативный)",
    "Аспект эффективности (позитивный)",
    "Аспект эффективности (негативный)",
    "Межличностный аспект (позитивный)",
    "Межличностный аспект (негативный)"
]

# Загрузка модели и токенизатора для Roberta
model_path = "./final_ruRoberta_multilabel_model"  # Путь к модели. Убедитесь, что он соответствует модели, обученной на основе Roberta.
tokenizer = RobertaTokenizer.from_pretrained(model_path)
model = RobertaForSequenceClassification.from_pretrained(model_path)
model.eval()

# Токенизация отзывов
inputs = tokenizer(
    reviews,
    truncation=True,
    padding=True,
    max_length=128,
    return_tensors="pt"
)

# Порог для активации метки (можно экспериментировать, например, 0.4)
threshold = 0.7

# Прогнозирование
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# Применяем сигмоиду и пороговое значение для получения бинарных меток
predictions = (torch.sigmoid(logits) > threshold).int().cpu().numpy()
print(predictions)

# Выводим результат для каждого отзыва
for review, pred in zip(reviews, predictions):
    active_labels = [label for label, flag in zip(label_names, pred) if flag == 1]
    print("Отзыв:", review)
    print("Предсказанные категории:", active_labels if active_labels else "Нет активных категорий")
    print()
